In [1]:
# Import required packages
import torch
import os
import numpy as np

from utils.solver import Solver, DEFAULT_SOLVER_PARAM_M7, DEFAULT_SOLVER_PARAM_M3
from utils.robot import get_robot, sample_P_path, sample_J_traj
from utils.utils import load_numpy, save_numpy, nearest_neighbor_F

In [2]:
robot = get_robot()
solver = Solver(robot=robot, solver_param=DEFAULT_SOLVER_PARAM_M7)

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [4]:
solver.path_following(load_time='', num_traj=100, shrink_ratio=0, enable_evaluation=True, enable_plot=False)

using shrink_ratio: 0
mkdir /home/luca/ikpflow/data/panda/trajectory/1111155930
/home/luca/ikpflow/data/panda/trajectory/1111155930/ load successfully.
move C to cuda
======509=(-0.04696768038025994)======
move C to cuda
======8=(-0.04554713016175273)======
move C to cuda
======481=(-0.046223948295776904)======
move C to cuda
======12=(-0.04279866414630872)======
move C to cuda
======216=(-0.04555061618414647)======
move C to cuda
======36=(-0.04279614664958055)======
move C to cuda
======111=(-0.10141172727165898)======
move C to cuda
======134=(0.06714431791164699)======
move C to cuda
======431=(-0.043699082197007945)======
move C to cuda
======281=(-0.04786337601067455)======
move C to cuda
======238=(-0.043374553520572276)======
move C to cuda
======434=(-0.047870439843639166)======
move C to cuda
======261=(0.06402353274220318)======
move C to cuda
======167=(-0.14107055541076485)======
move C to cuda
======425=(-0.045558611950691774)======
move C to cuda
======572=(0.03594320303

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
target_poses = load_numpy(file_path=traj_dir + 'ee_traj.npy')
# traj_dir, target_poses = get_target_poses(robot=panda)
exp_dir = traj_dir + f'exp_5_single_obj/'
data = {fn: [] for fn in file_names}

# -> unrefined solutions
mean_position_errors = np.zeros((num_ikflow_trails,))
mean_ang_errs = np.zeros((num_ikflow_trails,))
for ikflow_i in range(num_ikflow_trails):
    solutions, position_errors, _, _, _, _ = ik_solver.solve_n_poses(
        target_poses, 
        refine_solutions=False, 
        return_detailed=True)

    iksols = solutions.detach().cpu().numpy()
    df = eval_J_traj(robot=panda, J_traj=solutions.to('cpu'), P_path=target_poses, position_errors=position_errors)
    mean_position_errors[ikflow_i], mean_ang_errs[ikflow_i]  = df.mean().values
data['ikflow_l2'] = mean_position_errors
data['ikflow_ang'] = mean_ang_errs    


In [ ]:
df = pd.DataFrame(mean_position_errors)
df.describe()

In [ ]:
df = pd.DataFrame(mean_ang_errs)
df.describe()